In [23]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
#import simplejson as json
import requests
import time
from IPython.display import display

import pymysql


In [24]:
URI="dbbikes.cjk4ybuxtkwv.us-east-1.rds.amazonaws.com"
PORT="3306"
DB="dbbikes"
USER="cmcelduff"
PASSWORD="Tullamore1!"

In [25]:
#Connect to database
engine = create_engine("mysql+pymysql://{0}:{1}@{2}".format(USER, PASSWORD, URI), echo=True) 
connection = engine.connect()

2023-02-24 14:17:51,049 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2023-02-24 14:17:51,050 INFO sqlalchemy.engine.base.Engine {}
2023-02-24 14:17:51,183 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2023-02-24 14:17:51,184 INFO sqlalchemy.engine.base.Engine {}
2023-02-24 14:17:51,442 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2023-02-24 14:17:51,444 INFO sqlalchemy.engine.base.Engine {}
2023-02-24 14:17:51,681 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2023-02-24 14:17:51,681 INFO sqlalchemy.engine.base.Engine {}
2023-02-24 14:17:51,818 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2023-02-24 14:17:51,819 INFO sqlalchemy.engine.base.Engine {}
2023-02-24 14:17:51,954 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2023-02-24 14:17:51,955 INFO sqlalchemy.engine.base.E

In [26]:
sql = """
CREATE DATABASE IF NOT EXISTS dbikes;
"""
engine.execute(sql)

2023-02-24 14:17:53,469 INFO sqlalchemy.engine.base.Engine 
CREATE DATABASE IF NOT EXISTS dbikes;

2023-02-24 14:17:53,470 INFO sqlalchemy.engine.base.Engine {}
2023-02-24 14:17:53,625 INFO sqlalchemy.engine.base.Engine COMMIT


In [27]:
# command to check database is running
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

2023-02-24 14:17:53,923 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES;
2023-02-24 14:17:53,925 INFO sqlalchemy.engine.base.Engine {}
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('awsauthenticationplugin_max_backoff_delay', '2000')
('awsauthenticationplugin_max_retry_count', '3')
('awsauthenticationplugin_retry_delay', '250')
('back_log', '61')
('basedir', '/rdsdbbin/mysql-8.0.28.R4/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_di

In [28]:
sql_create_schema = "CREATE SCHEMA `dublin_bikes`;"
try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql_create_schema)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-02-24 14:17:54,386 INFO sqlalchemy.engine.base.Engine CREATE SCHEMA `dublin_bikes`;
2023-02-24 14:17:54,387 INFO sqlalchemy.engine.base.Engine {}
2023-02-24 14:17:54,551 INFO sqlalchemy.engine.base.Engine ROLLBACK
(pymysql.err.ProgrammingError) (1007, "Can't create database 'dublin_bikes'; database exists")
[SQL: CREATE SCHEMA `dublin_bikes`;]
(Background on this error at: http://sqlalche.me/e/f405)


In [29]:
# command to create table in mysql
sql_create_table = """
CREATE TABLE IF NOT EXISTS `dublin_bikes`.`weather` (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contact_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)
"""

try:
    #res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

    # tom test 
    # tom test 2 

2023-02-24 14:17:54,840 INFO sqlalchemy.engine.base.Engine 
CREATE DATABASE IF NOT EXISTS dbikes;

2023-02-24 14:17:54,841 INFO sqlalchemy.engine.base.Engine {}
2023-02-24 14:17:54,985 INFO sqlalchemy.engine.base.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [30]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations/"
DubBike_API = "7f06972a5ed335cf697379627fd13027274927c7"

def write_to_file(text):
    date_underscores = "{:%Y_%m_%d_%H_%M_%S}".format(datetime.datetime.now())
    with open(r"data\bikes_{}".format(date_underscores).replace(" ", "_"), "w") as f:
        f.write(text)

def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get("address"),int(station.get("banking")), int(station.get("bike_stands")), int(station.get("bonus")),station.get("contract_name"), station.get("name"), station.get("number"), station.get("position").get("lat"), station.get("position").get("lng"), station.get("status"))
        engine.execute("INSERT INTO `dublin_bikes`.`weather` values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", vals)
    return

def availability_to_db(text):
    stations = json.loads(text)
    #print(availability)
    for station in stations:
        #print(station.get("name"))
        #print(station.get("number"))
        #print(station.get("available_bikes"))
        #print(station.get("available_bike_stands"))
        #print(station.get("last_update"))
        vals = (int(station.get("number")), int(station.get("available_bikes")), int(station.get("available_bike_stands")), int(station.get("last_update")))
        engine.execute("INSERT INTO `dublin_bikes`.`availability` values(%s,%s,%s,%s)", vals)
    return
        
                                            
def main():
    print(os.path)
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS, params={"apiKey": DubBike_API, "contract" : NAME})
            print(r, now)
            write_to_file(r.text)
            stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5*60)
        except:
            print(traceback.format_exc())
            #if engine is None:
                #pass
        return

if __name__== "__main__":
    main()

<module 'posixpath' from '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/posixpath.py'>
Traceback (most recent call last):
  File "/var/folders/ng/dk0k01ws32sbbsg_81ftvw4w0000gn/T/ipykernel_15704/849790545.py", line 37, in main
    now = datetime.datetime.now()
NameError: name 'datetime' is not defined



In [31]:
NAME="Dublin"
STATIONS="https://api.jcdecaux.com/vls/v1/stations"
r = requests.get(STATIONS, params={"contract" : NAME, "apiKey": DubBike_API})

In [32]:
r.json()

[{'number': 42,
  'contract_name': 'dublin',
  'name': 'SMITHFIELD NORTH',
  'address': 'Smithfield North',
  'position': {'lat': 53.349562, 'lng': -6.278198},
  'banking': False,
  'bonus': False,
  'bike_stands': 30,
  'available_bike_stands': 20,
  'available_bikes': 10,
  'status': 'OPEN',
  'last_update': 1677248149000},
 {'number': 30,
  'contract_name': 'dublin',
  'name': 'PARNELL SQUARE NORTH',
  'address': 'Parnell Square North',
  'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526},
  'banking': False,
  'bonus': False,
  'bike_stands': 20,
  'available_bike_stands': 3,
  'available_bikes': 17,
  'status': 'OPEN',
  'last_update': 1677247973000},
 {'number': 54,
  'contract_name': 'dublin',
  'name': 'CLONMEL STREET',
  'address': 'Clonmel Street',
  'position': {'lat': 53.336021, 'lng': -6.26298},
  'banking': False,
  'bonus': False,
  'bike_stands': 33,
  'available_bike_stands': 24,
  'available_bikes': 9,
  'status': 'OPEN',
  'last_update': 1677248041000},
